<style>
    .rst-content blockquote {
        margin-left: 0px;
    }
    
    blockquote > div {
        margin: 1.5625em auto;
        padding: 20px 15px 1px;
        border-left: 0.2rem solid rgb(59, 136, 219);  
        border-radius: 0.2rem;
        box-shadow: 0 0.2rem 0.5rem rgb(0 0 0 / 5%), 0 0 0.0625rem rgb(0 0 0 / 10%);
    }
</style>

[View the runnable example on GitHub](https://github.com/intel-analytics/BigDL/tree/main/python/nano/tutorial/notebook/inference/pytorch/quantize_pytorch_inference_pot.ipynb)

# Quantize PyTorch Model for Inference using OpenVINO Post-training Optimization Tools

As Post-training Optimization Tools (POT) is provided by OpenVINO toolkit, OpenVINO acceleration will be enabled in the meantime when using POT for quantization. You can call `InferenceOptimizer.quantize` API with `accelerator='openvino'` to use POT for your PyTorch `nn.Module`. It only takes a few lines.

To quantize your model with POT, the following dependencies need to be installed first:

In [ ]:
# for BigDL-Nano
!pip install --pre --upgrade bigdl-nano[pytorch] # install the nightly-bulit version
# !source bigdl-nano-init

# for POT
!pip install openvino-dev

> 📝 **Note**
> 
> We recommend to run the commands above, especially `source bigdl-nano-init` before jupyter kernel is started, or some of the optimizations may not take effect.

> ⚠️ **Warning**
>
> Errors may occur when using `InferenceOptimizer.quantize(..., accelerator='openvino')` API in CentOS, becuase the latest version of `openvino-dev` is not supported in CentOS.

Let's take an [ResNet-18 model](https://pytorch.org/vision/main/models/generated/torchvision.models.resnet18.html) pretrained on ImageNet dataset and finetuned on [OxfordIIITPet dataset](https://pytorch.org/vision/main/generated/torchvision.datasets.OxfordIIITPet.html) as an example:

In [ ]:
# Define the finetune function
import torch
from torchvision import transforms
from torchvision.datasets import OxfordIIITPet
from torch.utils.data.dataloader import DataLoader
from torchvision.models import resnet18
from bigdl.nano.pytorch import Trainer
from torchmetrics import Accuracy


def finetune_pet_dataset(model_ft):

    train_transform = transforms.Compose([transforms.Resize(256),
                                          transforms.RandomCrop(224),
                                          transforms.RandomHorizontalFlip(),
                                          transforms.ColorJitter(brightness=.5, hue=.3),
                                          transforms.ToTensor(),
                                          transforms.Normalize([0.485, 0.456, 0.406],
                                                               [0.229, 0.224, 0.225])])
    val_transform = transforms.Compose([transforms.Resize(256),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406],
                                                             [0.229, 0.224, 0.225])])

    # apply data augmentation to the tarin_dataset
    train_dataset = OxfordIIITPet(root="/tmp/data",
                                  transform=train_transform,
                                  download=True)
    val_dataset = OxfordIIITPet(root="/tmp/data",
                                transform=val_transform)

    # obtain training indices that will be used for validation
    indices = torch.randperm(len(train_dataset))
    val_size = len(train_dataset) // 4
    train_dataset = torch.utils.data.Subset(train_dataset, indices[:-val_size])
    val_dataset = torch.utils.data.Subset(val_dataset, indices[-val_size:])

    # prepare data loaders
    train_dataloader = DataLoader(train_dataset, batch_size=32)
    val_dataloader = DataLoader(val_dataset, batch_size=32)

    num_ftrs = model_ft.fc.in_features

    # here the size of each output sample is set to 37
    model_ft.fc = torch.nn.Linear(num_ftrs, 37)
    loss_ft = torch.nn.CrossEntropyLoss()
    optimizer_ft = torch.optim.SGD(model_ft.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

    # compile our model with loss function, optimizer
    model = Trainer.compile(model_ft, loss_ft, optimizer_ft, metrics=[Accuracy()])
    trainer = Trainer(max_epochs=1)
    trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

    return model, train_dataset, val_dataset

In [ ]:
from torchvision.models import resnet18

model = resnet18(pretrained=True)
_, train_dataset, val_dataset = finetune_pet_dataset(model)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;_The full definition of function_ `finetune_pet_dataset` _could be found in the_ [runnable example](https://github.com/intel-analytics/BigDL/tree/main/python/nano/tutorial/notebook/inference/pytorch/quantize_pytorch_inference_pot.ipynb).

Then we set it in evaluation mode:

In [ ]:
model.eval()

To enable quantization using POT for inference, you could simply **import BigDL-Nano** `InferenceOptimizer`**, and use** `InferenceOptimizer` **to quantize your PyTorch model**:

In [ ]:
from bigdl.nano.pytorch import InferenceOptimizer

q_model = InferenceOptimizer.quantize(model,
                                      accelerator='openvino',
                                      calib_dataloader=DataLoader(train_dataset, batch_size=32))

> 📝 **Note**
> 
> For POT, only **static** post-training quantization is supported. So `calib_dataloader` (for calibration data) is always required when `accelerator='openvino'`. 
> 
> For `calib_dataloader`, batch size is not important as it intends to read 100 samples. And there could be no label in calibration data.
> 
> Please refer to [API documentation](https://bigdl.readthedocs.io/en/latest/doc/PythonAPI/Nano/pytorch.html#bigdl.nano.pytorch.InferenceOptimizer.quantize) for more information on `InferenceOptimizer.quantize`.

You could then do the normal inference steps with the quantized model:

In [ ]:
x = torch.stack([val_dataset[0][0], val_dataset[1][0]])
# use the quantized model here
y_hat = q_model(x)
predictions = y_hat.argmax(dim=1)
print(predictions)

> 📚 **Related Readings**
> 
> - [How to install BigDL-Nano](https://bigdl.readthedocs.io/en/latest/doc/Nano/Overview/nano.html#install)
> - [How to install BigDL-Nano in Google Colab](https://bigdl.readthedocs.io/en/latest/doc/Nano/Howto/install_in_colab.html)